In [ ]:
import numpy as np
from ipywidgets import widgets, interact

## Reimplementing Tensorboard Smoothing

* https://stackoverflow.com/questions/42281844/what-is-the-mathematics-behind-the-smoothing-parameter-in-tensorboards-scalar

* https://stackoverflow.com/questions/42869495/numpy-version-of-exponential-weighted-moving-average-equivalent-to-pandas-ewm


In [ ]:
import numpy as np
from ipywidgets import widgets, interact
import matplotlib.pyplot as plt

In [ ]:
def smooth(scalars, weight):  # Weight between 0 and 1
    if weight <= 0.0: # no smoothing
        return scalars
    last = scalars[0]  # First value in the plot (first timestep)
    smoothed = list()
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point  # Calculate smoothed value
        smoothed.append(smoothed_val)                        # Save it
        last = smoothed_val                                  # Anchor the last smoothed value

    return smoothed

def ewma(x, alpha):
    '''
    copied from https://stackoverflow.com/questions/42869495/numpy-version-of-exponential-weighted-moving-average-equivalent-to-pandas-ewm
    Returns the exponentially weighted moving average of x.

    Parameters:
    -----------
    x : array-like
    alpha : float {0 <= alpha <= 1}

    Returns:
    --------
    ewma: numpy array
          the exponentially weighted moving average
    '''
    # Coerce x to an array
    if alpha <= 0.0:
        return x
    x = np.array(x)
    n = x.size

    # Create an initial weight matrix of (1-alpha), and a matrix of powers
    # to raise the weights by
    w0 = np.ones(shape=(n,n)) * (alpha)
    p = np.vstack([np.arange(i,i-n,-1) for i in range(n)])

    # Create the weight matrix
    w = np.tril(w0**p,0)

    # Calculate the ewma
    return np.dot(w, x[::np.newaxis]) / w.sum(axis=1)


implementations = {
    "tensorboard" : smooth,
    "ewma" : ewma
}

In [ ]:
def change_fontsize(ax, fs):
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(fs)
    return ax
    
def change_linewidth(ax, lw=3):
    for item in ax.lines:
        item.set_linewidth(lw)
    return ax

def change_legend_linewidth(ax, lw=2.0):
    leg = ax.legend_
    for legobj in leg.legendHandles:
        legobj.set_linewidth(lw)

In [ ]:
x = np.linspace(0, 1, 20)
y = x.copy()
y += np.random.randn(*y.shape) / 10
def _plot(alpha):
    plt.close()
    fig, ax = plt.subplots(1, 1, figsize=(8,8))
    for label, func in implementations.items():
        y_smooth = func(y, alpha)
        plt.plot(x, y_smooth, label=label)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.grid(True)
    fig.tight_layout()
    ax.legend(frameon=True, fontsize=24, loc="upper left")
    change_fontsize(ax, 16)
    change_linewidth(ax, 3)#
    change_legend_linewidth(ax, 3)
alpha = widgets.FloatSlider(value=0.0, min=0.00, max=1, step=0.1)
interact(_plot, alpha=alpha)